In [4]:
import pandas as pd
import os

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, mean_squared_error, r2_score
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

In [7]:
all_data = pd.read_csv("model_data.csv",index_col=[0,1],header=[0,1,2])

In [8]:
all_data.index.rename(['track_id', 'genre_top'], inplace=True)

In [10]:
all_data.reset_index(1,inplace=True)

In [11]:
all_data['genre_top'].value_counts()

genre_top
Experimental           24461
Rock                   23084
Electronic             21794
Hip-Hop                 6465
Pop                     6425
Folk                    6170
Instrumental            5690
International           2690
Classical               1675
Spoken                  1335
Country                 1207
Soul-RnB                 941
Jazz                     841
Old-Time / Historic      658
Easy Listening           460
Blues                    447
Name: count, dtype: int64

In [13]:
all_data.dropna(inplace=True)

In [14]:
all_data

genre_top chroma_cens                                          \
                      kurtosis                                           
                            01        02        03        04        05   
track_id                                                                 
2          Hip-Hop    7.180653  5.230309  0.249321  1.347620  1.482478   
3          Hip-Hop    1.888963  0.760539  0.345297  2.295201  1.654031   
5          Hip-Hop    0.527563 -0.077654 -0.279610  0.685883  1.937570   
10             Pop    3.702245 -0.291193  2.196742 -0.234449  1.367364   
20             Pop   -0.193837 -0.198527  0.201546  0.258556  0.775204   
...            ...         ...       ...       ...       ...       ...   
155316        Rock   -0.490129  0.463834  2.321970 -0.084352  1.662914   
155317        Rock   -0.461559 -0.229601 -0.496632 -0.422033  0.130612   
155318        Rock    0.552473 -0.110498 -0.532014  0.263131 -0.224011   
155319        Rock   -0.176901  0.187208 -0.050664  0.368843  0.066005   
155320     Country    0.489665  1.862421  0.854461 -0.103666 -0.249835   

                                                  ...  mel_spec            \
                                                  ...       std             
                06        07        08        09  ...        90        91   
track_id                                          ...                       
2         0.531371  1.481593  2.691455  0.866868  ...  0.005500  0.004066   
3         0.067592  1.366848  1.054094  0.108103  ...  0.014708  0.019320   
5         0.880839 -0.923192 -0.927232  0.666617  ...  0.000010  0.000009   
10        0.998411  1.770694  1.604566  0.521217  ...  0.018939  0.015547   
20        0.084794 -0.289294 -0.816410  0.043851  ...  0.008917  0.008339   
...            ...       ...       ...       ...  ...       ...       ...   
155316    2.115189 -0.237794  5.695442  0.830353  ...  0.001007  0.000663   
155317   -0.263825 -0.628103 -0.082687 -0.229483  ...  0.002338  0.001594   
155318   -0.530972  1.713526  1.418444  1.325197  ...  0.003115  0.002223   
155319   -0.857354 -0.780860  0.626281 -0.630938  ...  0.002461  0.001665   
155320    0.360283 -0.366701  0.033578 -0.834606  ...  0.015526  0.013577   

                                                                      \
                                                                       
                92        93        94        95        96        97   
track_id                                                               
2         0.003965  0.002347  0.001663  0.002632  0.001452  0.000114   
3         0.078528  0.072792  0.069122  0.013616  0.008447  0.001303   
5         0.000008  0.000006  0.000005  0.000007  0.000004  0.000002   
10        0.010535  0.006567  0.006368  0.003965  0.003788  0.002343   
20        0.006849  0.006172  0.005579  0.004760  0.004246  0.004115   
...            ...       ...       ...       ...       ...       ...   
155316    0.000429  0.000345  0.000208  0.000123  0.000121  0.000038   
155317    0.001051  0.000818  0.000603  0.000557  0.000391  0.000214   
155318    0.001846  0.001351  0.000899  0.000688  0.000458  0.000194   
155319    0.001135  0.000730  0.000473  0.000313  0.000198  0.000103   
155320    0.011346  0.010214  0.008648  0.007865  0.006310  0.005185   

                              
                              
                98        99  
track_id                      
2         0.000002  0.000002  
3         0.000003  0.000002  
5         0.000002  0.000001  
10        0.001213  0.000316  
20        0.003781  0.003704  
...            ...       ...  
155316    0.000002  0.000001  
155317    0.000067  0.000098  
155318    0.000042  0.000019  
155319    0.000006  0.000005  
155320    0.003995  0.003733  

[104337 rows x 1415 columns]

In [328]:
new_pop_data = pd.read_csv("new_data_only_pop.csv",header=[0,1,2],index_col = [0,1])

In [336]:
new_pop_data.index.rename(["track_id","genre_top"],inplace=True)

In [340]:
new_pop_data.reset_index([1],inplace=True)

In [342]:
pd.concat([all_data,new_pop_data])

genre_top chroma_cens                                          \
                           kurtosis                                           
                                 01        02        03        04        05   
track_id                                                                      
2               Hip-Hop    7.180653  5.230309  0.249321  1.347620  1.482478   
3               Hip-Hop    1.888963  0.760539  0.345297  2.295201  1.654031   
5               Hip-Hop    0.527563 -0.077654 -0.279610  0.685883  1.937570   
10                  Pop    3.702245 -0.291193  2.196742 -0.234449  1.367364   
20                  Pop   -0.193837 -0.198527  0.201546  0.258556  0.775204   
...                 ...         ...       ...       ...       ...       ...   
pop.00095.wav       Pop   -0.174348 -0.659997 -0.497875 -0.052841  1.289440   
pop.00096.wav       Pop    1.376895 -0.397864 -1.005848 -1.076321 -0.643594   
pop.00097.wav       Pop    1.083995 -0.387100 -0.098440 -0.264428  0.444005   
pop.00098.wav       Pop    5.680593 -0.039087 -0.569500 -0.587668 -0.806135   
pop.00099.wav       Pop   -0.717607 -0.297106  0.099448 -0.879806 -1.128139   

                                                       ...  mel_spec  \
                                                       ...       std   
                     06        07        08        09  ...        90   
track_id                                               ...             
2              0.531371  1.481593  2.691455  0.866868  ...  0.005500   
3              0.067592  1.366848  1.054094  0.108103  ...  0.014708   
5              0.880839 -0.923192 -0.927232  0.666617  ...  0.000010   
10             0.998411  1.770694  1.604566  0.521217  ...  0.018939   
20             0.084794 -0.289294 -0.816410  0.043851  ...  0.008917   
...                 ...       ...       ...       ...  ...       ...   
pop.00095.wav  0.939102  0.375670 -0.792472  4.272242  ...  0.100854   
pop.00096.wav -1.141642 -1.157031 -0.838738  4.966903  ...  0.138672   
pop.00097.wav  0.379076  7.424728  6.734740  5.900572  ...  0.084242   
pop.00098.wav -0.410198 -0.466205 -1.326072  4.206135  ...  0.115378   
pop.00099.wav  0.244769  1.155673  0.183324 -0.036082  ...  0.060146   

                                                                           \
                                                                            
                     91        92        93        94        95        96   
track_id                                                                    
2              0.004066  0.003965  0.002347  0.001663  0.002632  0.001452   
3              0.019320  0.078528  0.072792  0.069122  0.013616  0.008447   
5              0.000009  0.000008  0.000006  0.000005  0.000007  0.000004   
10             0.015547  0.010535  0.006567  0.006368  0.003965  0.003788   
20             0.008339  0.006849  0.006172  0.005579  0.004760  0.004246   
...                 ...       ...       ...       ...       ...       ...   
pop.00095.wav  0.102026  0.104845  0.103215  0.114521  0.106196  0.116742   
pop.00096.wav  0.141954  0.138962  0.127535  0.124761  0.127113  0.146227   
pop.00097.wav  0.089052  0.101671  0.112145  0.106398  0.108568  0.101924   
pop.00098.wav  0.106202  0.099811  0.100815  0.104626  0.091387  0.087154   
pop.00099.wav  0.061531  0.067899  0.069413  0.063453  0.060193  0.051492   

                                             
                                             
                     97        98        99  
track_id                                     
2              0.000114  0.000002  0.000002  
3              0.001303  0.000003  0.000002  
5              0.000002  0.000002  0.000001  
10             0.002343  0.001213  0.000316  
20             0.004115  0.003781  0.003704  
...                 ...       ...       ...  
pop.00095.wav  0.126092  0.109812  0.103226  
pop.00096.wav  0.145995  0.124675  0.120713  
pop.00097.wav  0.084924  0.078212  0.07

In [344]:
selected_model_data = all_data[all_data['genre_top'].isin(["Rock","Electronic","Pop","Hip-Hop","Folk"])]

In [346]:
selected_model_data

genre_top chroma_cens                                          \
                      kurtosis                                           
                            01        02        03        04        05   
track_id                                                                 
2          Hip-Hop    7.180653  5.230309  0.249321  1.347620  1.482478   
3          Hip-Hop    1.888963  0.760539  0.345297  2.295201  1.654031   
5          Hip-Hop    0.527563 -0.077654 -0.279610  0.685883  1.937570   
10             Pop    3.702245 -0.291193  2.196742 -0.234449  1.367364   
20             Pop   -0.193837 -0.198527  0.201546  0.258556  0.775204   
...            ...         ...       ...       ...       ...       ...   
155315        Rock   -0.319850  0.653044  1.300360 -0.606061 -1.109302   
155316        Rock   -0.490129  0.463834  2.321970 -0.084352  1.662914   
155317        Rock   -0.461559 -0.229601 -0.496632 -0.422033  0.130612   
155318        Rock    0.552473 -0.110498 -0.532014  0.263131 -0.224011   
155319        Rock   -0.176901  0.187208 -0.050664  0.368843  0.066005   

                                                  ...  mel_spec            \
                                                  ...       std             
                06        07        08        09  ...        90        91   
track_id                                          ...                       
2         0.531371  1.481593  2.691455  0.866868  ...  0.005500  0.004066   
3         0.067592  1.366848  1.054094  0.108103  ...  0.014708  0.019320   
5         0.880839 -0.923192 -0.927232  0.666617  ...  0.000010  0.000009   
10        0.998411  1.770694  1.604566  0.521217  ...  0.018939  0.015547   
20        0.084794 -0.289294 -0.816410  0.043851  ...  0.008917  0.008339   
...            ...       ...       ...       ...  ...       ...       ...   
155315   -1.006683  0.059643  0.472580 -0.856436  ...  0.000922  0.000430   
155316    2.115189 -0.237794  5.695442  0.830353  ...  0.001007  0.000663   
155317   -0.263825 -0.628103 -0.082687 -0.229483  ...  0.002338  0.001594   
155318   -0.530972  1.713526  1.418444  1.325197  ...  0.003115  0.002223   
155319   -0.857354 -0.780860  0.626281 -0.630938  ...  0.002461  0.001665   

                                                                      \
                                                                       
                92        93        94        95        96        97   
track_id                                                               
2         0.003965  0.002347  0.001663  0.002632  0.001452  0.000114   
3         0.078528  0.072792  0.069122  0.013616  0.008447  0.001303   
5         0.000008  0.000006  0.000005  0.000007  0.000004  0.000002   
10        0.010535  0.006567  0.006368  0.003965  0.003788  0.002343   
20        0.006849  0.006172  0.005579  0.004760  0.004246  0.004115   
...            ...       ...       ...       ...       ...       ...   
155315    0.000138  0.000019  0.000003  0.000004  0.000004  0.000003   
155316    0.000429  0.000345  0.000208  0.000123  0.000121  0.000038   
155317    0.001051  0.000818  0.000603  0.000557  0.000391  0.000214   
155318    0.001846  0.001351  0.000899  0.000688  0.000458  0.000194   
155319    0.001135  0.000730  0.000473  0.000313  0.000198  0.000103   

                              
                              
                98        99  
track_id                      
2         0.000002  0.000002  
3         0.000003  0.000002  
5         0.000002  0.000001  
10        0.001213  0.000316  
20        0.003781  0.003704  
...            ...       ...  
155315    0.000003  0.000002  
155316    0.000002  0.000001  
155317    0.000067  0.000098  
155318    0.000042  0.000019  
155319    0.000006  0.000005  

[63934 rows x 1415 columns]

In [348]:
dictG = {"Rock":0,"Electronic":1,"Pop":2,"Hip-Hop":3,"Folk":4}

In [608]:
X = selected_model_data.drop(["genre_top"],axis=1)
y = selected_model_data['genre_top'].replace(dictG)

/var/folders/gv/571nb6gs4h58lv0gw0__gy280000gn/T/ipykernel_38766/3008620335.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X = selected_model_data.drop(["genre_top"],axis=1)
/var/folders/gv/571nb6gs4h58lv0gw0__gy280000gn/T/ipykernel_38766/3008620335.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = selected_model_data['genre_top'].replace(dictG)


In [610]:
X.shape

(63934, 1414)

In [564]:
from sklearn.preprocessing import StandardScaler
scaler_standard = StandardScaler()
X = scaler_standard.fit_transform(X)

In [566]:
X

array([[-0.04814935, -0.27327681, -0.34798609, ...,  0.01790592,
         0.01918717,  0.27410291],
       [-0.01378304, -0.2990397 , -0.30993121, ...,  0.04205953,
         0.23861899,  1.18242837],
       [-0.14440378, -0.1147336 , -0.19830629, ...,  0.20010919,
         0.90524203,  0.23646261],
       ...,
       [-0.3462169 , -0.23511807, -0.19764029, ..., -0.95929661,
        -0.96988189, -1.11942986],
       [-0.30395615, -0.57665848, -0.12259865, ..., -1.00058955,
        -0.87743352, -0.99954596],
       [ 0.05399778,  0.12144351,  0.63313869, ..., -1.1805701 ,
        -1.197915  , -1.12291371]])

In [568]:
y.shape

(63934,)

In [570]:
x_train, x_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3)
x_val, x_test, y_val, y_test = train_test_split(
    x_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [572]:
y.value_counts()

genre_top
0    23083
1    21791
3     6465
2     6425
4     6170
Name: count, dtype: int64

In [575]:
# y_train_resampled.value_counts()

In [577]:
model = XGBClassifier(subsample= 0.8, reg_lambda=1.5, n_estimators= 500, max_depth= 5, learning_rate= 0.1, gamma= 0, colsample_bytree= 0.7)

In [579]:
# from sklearn.model_selection import RandomizedSearchCV
# param_grid = {
#     'n_estimators': [100, 300, 500],
#     'max_depth': [3, 5, 7],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'subsample': [0.7, 0.8, 1.0],
#     'colsample_bytree': [0.7, 0.8, 1.0],
#     'gamma': [0, 0.1, 0.2],
#     'reg_lambda': [1, 1.5, 2]
# }
# random_search = RandomizedSearchCV(
#     estimator=model,
#     param_distributions=param_grid,
#     n_iter=30,            # number of random combinations to try
#     scoring='accuracy',
#     cv=3,
#     verbose=2,
#     random_state=42,
#     n_jobs=-1
# )

# random_search.fit(x_train, y_train)

# print("Best Random Search Parameters:", random_search.best_params_)
# best_xgb_random = random_search.best_estimator_


In [581]:
# best_xgb_random

In [583]:
# y_train_resampled

In [585]:
# y_train.value_counts()

In [587]:
# y_train_resampled = y_train_resampled.replace(dictG)

In [589]:
# from imblearn.over_sampling import SMOTE

# smote = SMOTE(sampling_strategy='auto', random_state=42)
# x_train, y_train = smote.fit_resample(x_train, y_train)
from sklearn.utils.class_weight import compute_class_weight
# import numpy as np

# # Suppose y_train has classes like [0, 1, 2, 3]
# classes = np.unique(y_train)
# class_weights = compute_class_weight('balanced', classes=classes, y=y_train)

# print(class_weights)  # one weight for each class

# weights = np.array([class_weights[i] for i in y_train])
import numpy as np
classes = np.unique(y_train)
class_weights = compute_class_weight('balanced', classes=classes, y=y_train)

# Now create sample weights
sample_weights = np.array([class_weights[label] for label in y_train])

In [591]:
eval_set = [(x_train, y_train), (x_val, y_val)]

In [593]:
model.fit(x_train,y_train,sample_weight=sample_weights,eval_set=[(x_val, y_val)],verbose=True)

[0]	validation_0-mlogloss:1.57009
[1]	validation_0-mlogloss:1.53426
[2]	validation_0-mlogloss:1.50317
[3]	validation_0-mlogloss:1.47523
[4]	validation_0-mlogloss:1.45041
[5]	validation_0-mlogloss:1.42816
[6]	validation_0-mlogloss:1.40827
[7]	validation_0-mlogloss:1.39068
[8]	validation_0-mlogloss:1.37405
[9]	validation_0-mlogloss:1.35851
[10]	validation_0-mlogloss:1.34442
[11]	validation_0-mlogloss:1.33147
[12]	validation_0-mlogloss:1.31951
[13]	validation_0-mlogloss:1.30845
[14]	validation_0-mlogloss:1.29815
[15]	validation_0-mlogloss:1.28840
[16]	validation_0-mlogloss:1.27966
[17]	validation_0-mlogloss:1.27163
[18]	validation_0-mlogloss:1.26396
[19]	validation_0-mlogloss:1.25703
[20]	validation_0-mlogloss:1.25010
[21]	validation_0-mlogloss:1.24360
[22]	validation_0-mlogloss:1.23746
[23]	validation_0-mlogloss:1.23178
[24]	validation_0-mlogloss:1.22665
[25]	validation_0-mlogloss:1.22128
[26]	validation_0-mlogloss:1.21630
[27]	validation_0-mlogloss:1.21182
[28]	validation_0-mlogloss:1.2

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=0, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=None, num_parallel_tree=None, ...)

In [594]:
y_pred=model.predict(x_test)

In [595]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.68      0.72      3445
           1       0.70      0.69      0.69      3286
           2       0.28      0.26      0.27       950
           3       0.49      0.59      0.53       967
           4       0.52      0.65      0.58       943

    accuracy                           0.63      9591
   macro avg       0.55      0.57      0.56      9591
weighted avg       0.64      0.63      0.63      9591



In [425]:
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(score_func=f_classif, k=1000)  # select top 50 features
X_train_selected = selector.fit_transform(x_train, y_train)
X_test_selected = selector.transform(x_test)
X_val_selected = selector.transform(x_val)

In [426]:
# import matplotlib.pyplot as plt

# # Plot top 50 features
# top_n = 50

# plt.figure(figsize=(10, 12))
# plt.barh(feat_importances['feature'][:top_n], feat_importances['importance'][:top_n])
# plt.gca().invert_yaxis()  # Most important on top
# plt.xlabel('Importance')
# plt.title(f'Top {top_n} Feature Importances')
# plt.show()

In [436]:
best_xgb_random.fit(X_train_selected,y_train,sample_weight=sample_weights,eval_set=[(X_val_selected, y_val)],verbose=True)

[0]	validation_0-mlogloss:1.56132
[1]	validation_0-mlogloss:1.51920
[2]	validation_0-mlogloss:1.48154
[3]	validation_0-mlogloss:1.44850
[4]	validation_0-mlogloss:1.41932
[5]	validation_0-mlogloss:1.39339
[6]	validation_0-mlogloss:1.36929
[7]	validation_0-mlogloss:1.34718
[8]	validation_0-mlogloss:1.32707
[9]	validation_0-mlogloss:1.30856
[10]	validation_0-mlogloss:1.29222
[11]	validation_0-mlogloss:1.27706
[12]	validation_0-mlogloss:1.26229
[13]	validation_0-mlogloss:1.24901
[14]	validation_0-mlogloss:1.23569
[15]	validation_0-mlogloss:1.22375
[16]	validation_0-mlogloss:1.21208
[17]	validation_0-mlogloss:1.20157
[18]	validation_0-mlogloss:1.19218
[19]	validation_0-mlogloss:1.18312
[20]	validation_0-mlogloss:1.17515
[21]	validation_0-mlogloss:1.16746
[22]	validation_0-mlogloss:1.15952
[23]	validation_0-mlogloss:1.15268
[24]	validation_0-mlogloss:1.14549
[25]	validation_0-mlogloss:1.13929
[26]	validation_0-mlogloss:1.13305
[27]	validation_0-mlogloss:1.12681
[28]	validation_0-mlogloss:1.1

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=0, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=None, num_parallel_tree=None, ...)

In [437]:
y_test_pred = best_xgb_random.predict(X_test_selected)

In [438]:
print(classification_report(y_test_pred, y_test))

              precision    recall  f1-score   support

           0       0.72      0.79      0.76      3132
           1       0.80      0.74      0.77      3514
           2       0.36      0.40      0.38       877
           3       0.64      0.64      0.64      1000
           4       0.69      0.60      0.64      1068

    accuracy                           0.70      9591
   macro avg       0.64      0.63      0.64      9591
weighted avg       0.70      0.70      0.70      9591



In [638]:
we_will_rock_you = pd.read_csv("we_will_rock_you.csv",header=[0,1,2],index_col=[0])

In [640]:
we_will_rock_you

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
0              2.14157  0.411554 -0.419062  0.632222  0.564811  0.001753   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
0           0.424778  2.937335  0.471645  0.067198  ...  0.102356  0.034659   

feature                    zcr                                               \
statistics            kurtosis       max      mean    median  min      skew   
number           06         01        01        01        01   01        01   
0           0.03353  33.296363  0.554199  0.062411  0.050293  0.0  5.113267   

feature               
statistics       std  
number            01  
0           0.060578  

[1 rows x 1414 columns]

In [642]:
best_xgb_random.predict_proba(selector.transform(we_will_rock_you))

array([[0.31377733, 0.5654845 , 0.02500724, 0.09083951, 0.00489141]],
      dtype=float32)